# Bibliografía

https://towardsdatascience.com/data-visualization-with-bokeh-in-python-part-ii-interactions-a4cf994e2512


# Carga de librerías

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
import seaborn as sns
from sklearn.metrics import mean_squared_error
from bokeh.plotting import figure
from bokeh.models import CustomJS, Slider, layouts, ColumnDataSource
from bokeh.io import output_notebook, show
from bokeh.layouts import row, column, gridplot
from bokeh.models.widgets import Button, RadioButtonGroup, Select, Slider

# Para que se muestren las grafícas del bokeh en el notebook
from bokeh.sampledata.sea_surface_temperature import sea_surface_temperature
output_notebook()

Loading BokehJS ...

In [3]:
class Ansete(object):
    def __init__(self, file):
        self.df = pd.read_csv(file)
        # Convierto la columna de fecha a datetime
        self.df['fecha'] = pd.to_datetime(self.df['fecha'])

        # Creo una columna con la diferencia entre el valor real y la predicción
        self.df['diff'] = self.df['ocu_real'] - self.df['ocu_pred']

        # Creo una columna con el error relativo correspondiente a los datos reales y las predicciones
        self.df['error_relativo'] = np.abs(self.df['ocu_pred'] - self.df['ocu_real']) / self.df['ocu_real'] * 100

        self.df_test = self.select_test()
        
        # Por defecto va a mostrar todos los valores
        self.df_show = self.df

    def show_graph(self, source):
        """
        Muestra los gráficos superpuestos para los datos reales y las predicciones. En naranja se encuentran los datos
        reales y en azul las predicciónes realizadas para ese instante
        """
        fig = figure(x_axis_type='datetime', title='Real Vs Predicción', x_axis_label='x', y_axis_label='ocupación',
                     y_range=[-10, 30], plot_width=925, plot_height=325)
        figline_real = fig.line('fecha', 'ocu_real', source=source, color='orange', line_width=1)
        figline_pred = fig.line('fecha', 'ocu_pred', source=source, color='blue', line_width=1)
        figline_med = fig.line('fecha', 'ocu_medida', source=source, color='green', line_width=1)
        return fig

    def show_diff_bokeh(self, source):
        """
        Representa las diferencias entre los valores de ocupación real y predicción en cada instante
        """
        fig = figure(x_axis_type='datetime', title='diferencia entre real y predicción', x_axis_label='x',
                     y_axis_label='diferencia', plot_width=925, plot_height=325)
        figline = fig.line('fecha', 'diff', source=source, color='orange', line_width=2)
        return fig

    def show_diff_relativo_bokeh(self, source):
        """
        Muestra el error relativo entre las medidas de la ocupación real y la predicción realizada en un instante
        """
        fig = figure(x_axis_type='datetime', title='error relativo', x_axis_label='x', y_axis_label='% error',
                     plot_width=925, plot_height=325)
        figline = fig.line('fecha', 'error_relativo', source=source, color='orange', line_width=1)
        return fig

    def select_test(self):
        """
        Datos del conjunto de test
        """
        df2 = self.df[self.df['fecha'] > '2019-11-18 00:00:00']
        df2 = df2.dropna()
        return df2
    
    def filter_data(self, data):
        df = self.df[self.df['fecha'] > data]
        return df

    def rmse(self, df):
        """
        Calcula el error cuadrático medios de los datos reales y la predicción para todos los clusters
        """
        rmse = {}
        print("Iniciando")
        df = df.dropna()
        for i in range(0, 200):
            df_mean_cluster = df[df['cluster'] == i]
            y_real = df_mean_cluster['ocu_real'].values
            y_pred = df_mean_cluster['ocu_pred'].values
            rms = mean_squared_error(y_real, y_pred)
            rmse[i] = np.sqrt(rms)
        return rmse
    
    def set_df(self, df):
        self.df_show = df
        
    def show_all(self, doc):
        """
        # creo la grid
        # grid = gridplot([[slider, None],[fig_all, fig_diff], [fig_err, None]])
        # show(grid)
        Muestra tres gráficas: las diferencias, los errores y la representación de los datos, así como también muestra
        el valor del error cuadrático medio obtenido para los datos
        """
        df_mean_cluster = self.df_show
        print(df_mean_cluster.head())
        df_filter = df_mean_cluster.dropna()

        # Muestro por defecto el cluster 0
        df_graph = df_filter[df_filter['cluster'] == 0]

        # Selecciono la fuente de datos
        source = ColumnDataSource(data=df_graph)

        fig_all = self.show_graph(source)
        fig_diff = self.show_diff_bokeh(source)
        fig_diff.x_range = fig_all.x_range
        fig_err = self.show_diff_relativo_bokeh(source)
        fig_err.x_range = fig_all.x_range

        def callback(attr, old, new):
            data = df_filter[df_filter['cluster'] == new]
            source.data = ColumnDataSource(data=data).data

        slider = Slider(start=0, end=199, value=0, step=1, title="Cluster")
        slider.on_change('value', callback)
        doc.add_root(column(slider, fig_all, fig_diff, fig_err))

        y_real = df_filter['ocu_real'].values
        y_pred = df_filter['ocu_pred'].values
        rms = mean_squared_error(y_real, y_pred)
        print("rmse: ", np.sqrt(rms))
        


## Carga de datasets

In [4]:
predict = Ansete(file='predict.csv')

In [5]:
filter_data = predict.filter_data(data='2019-11-27 00:04:26')

In [6]:
filter_data.head()

,id_pred,cluster,fecha,ocu_real,ocu_pred,ocu_medida,diff,error_relativo
48886,48887,0,2019-11-27 00:07:58,2.227273,5.743430,1.568966,-3.516157,157.868277
48887,48888,1,2019-11-27 00:07:58,1.450000,4.175496,3.583333,-2.725496,187.965264
48888,48889,2,2019-11-27 00:07:58,2.500000,2.603840,1.444444,-0.103840,4.153589
48889,48890,3,2019-11-27 00:07:58,1.043478,1.064703,1.458333,-0.021225,2.034047
48890,48891,4,2019-11-27 00:07:58,2.222222,2.806649,1.462963,-0.584427,26.299218


In [7]:
predict.set_df(filter_data)

In [8]:
show(predict.show_all)

       id_pred  cluster               fecha  ocu_real  ocu_pred  ocu_medida  \
48886    48887        0 2019-11-27 00:07:58  2.227273  5.743430    1.568966   
48887    48888        1 2019-11-27 00:07:58  1.450000  4.175496    3.583333   
48888    48889        2 2019-11-27 00:07:58  2.500000  2.603840    1.444444   
48889    48890        3 2019-11-27 00:07:58  1.043478  1.064703    1.458333   
48890    48891        4 2019-11-27 00:07:58  2.222222  2.806649    1.462963   

           diff  error_relativo  
48886 -3.516157      157.868277  
48887 -2.725496      187.965264  
48888 -0.103840        4.153589  
48889 -0.021225        2.034047  
48890 -0.584427       26.299218  
rmse:  1.7426284801112375
